In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd
import numpy as np
import Sastrawi

ModuleNotFoundError: ignored

In [2]:
!pip install PySastrawi

     |████████████████████████████████| 215kB 20.8MB/s 


In [3]:
#DATASET SOURCE FROM https://github.com/dbrehmer/Knowself/blob/master/data/mypersonality/essays.csv"
#DATASET USED IN THIS NOTEBOOK IS DATASET FROM THE SOURCE THAT HAS BEEN TRANSLATED TO BAHASA INDONESIA USING GOOGLE TRANSLATE

DATASET_URL = "https://raw.githubusercontent.com/lazuardi100/Hexa-Engineer/ML/Dataset/dataset.csv"
df = pd.read_csv(DATASET_URL, sep =',')

df.head()

,#AUTHID,TTEXT,cEXT,cNEU,cAGR,cCON,cOPN
0,1997_504851.txt,"Nah, sekarang saya baru saja bangun dari tidur...",n,y,y,n,y
1,1997_605191.txt,"Nah, di sini kita pergi dengan arus kesadaran ...",n,n,y,n,n
2,1997_687252.txt,Keyboard terbuka dan tombol untuk mendorong. H...,n,y,n,y,y
3,1997_568848.txt,Aku tidak percaya itu! Ini benar-benar terjadi...,y,n,y,y,n
4,1997_688160.txt,"Nah, di sini aku pergi dengan aliran tua yang ...",y,n,y,n,y


In [4]:
def changeLabel(labels):
  for index, values in enumerate(labels.values):
    if values == 'n':
      labels[index] = 0
    else:
      labels[index] = 1
  return labels

changeLabel(df['cEXT'])
changeLabel(df['cNEU'])
changeLabel(df['cAGR'])
changeLabel(df['cCON'])
changeLabel(df['cOPN'])

0       1
1       0
2       1
3       0
4       1
       ..
2462    0
2463    1
2464    0
2465    1
2466    1
Name: cOPN, Length: 2467, dtype: object

In [5]:
#Text Preprocessing referenced from https://github.com/ksnugroho/basic-text-preprocessing/blob/master/text-preprocessing.ipynb
import string
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

def preprocess_text(texts):
  for text in texts:
    #lowercase all character in the text
    text = text.lower()
    #remove punctuation
    text = text.translate(str.maketrans("","",string.punctuation))
    #remove leading and trailing whitespace
    text = text.strip()
    #remove StopWord
    stopword = StopWordRemoverFactory().create_stop_word_remover()
    text = stopword.remove(text)
    #stemming
    stemmer = StemmerFactory().create_stemmer()
    text = stemmer.stem(text)
  
  return texts

df['TTEXT'] = preprocess_text(df['TTEXT'])

In [6]:
df = df.sample(frac=1)
train_dataset, val_dataset, test_dataset = np.split(df, [int(.95 * len(df)), int(.975 * len(df))])

In [7]:
train_data = train_dataset['TTEXT']
train_label_ext  = train_dataset['cEXT']
train_label_neu  = train_dataset['cNEU']
train_label_agr  = train_dataset['cAGR']
train_label_con  = train_dataset['cCON']
train_label_opn  = train_dataset['cOPN']

val_data = val_dataset['TTEXT']
val_label_ext  = val_dataset['cEXT']
val_label_neu  = val_dataset['cNEU']
val_label_agr  = val_dataset['cAGR']
val_label_con  = val_dataset['cCON']
val_label_opn  = val_dataset['cOPN']

test_data = test_dataset['TTEXT']
test_label_ext  = test_dataset['cEXT']
test_label_neu  = test_dataset['cNEU']
test_label_agr  = test_dataset['cAGR']
test_label_con  = test_dataset['cCON']
test_label_opn  = test_dataset['cOPN']

In [8]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#Tried to use parameter and model from the NLP Course by Deeplearning.ai
vocab_size = 1000
embedding_dim = 16
max_length = 120
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_data)

training_sequences = tokenizer.texts_to_sequences(train_data)
training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

val_sequences = tokenizer.texts_to_sequences(val_data)
val_padded = pad_sequences(val_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

test_sequences = tokenizer.texts_to_sequences(test_data)
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)

In [9]:
train_dataset_ext = tf.data.Dataset.from_tensor_slices((train_data.values, train_label_ext.values.astype(dtype=np.float32)))
train_dataset_neu = tf.data.Dataset.from_tensor_slices((train_data.values, train_label_neu.values.astype(dtype=np.float32)))
train_dataset_agr = tf.data.Dataset.from_tensor_slices((train_data.values, train_label_agr.values.astype(dtype=np.float32)))
train_dataset_con = tf.data.Dataset.from_tensor_slices((train_data.values, train_label_con.values.astype(dtype=np.float32)))
train_dataset_opn = tf.data.Dataset.from_tensor_slices((train_data.values, train_label_opn.values.astype(dtype=np.float32)))

val_dataset_ext = tf.data.Dataset.from_tensor_slices((val_data.values, val_label_ext.values.astype(dtype=np.float32)))
val_dataset_neu = tf.data.Dataset.from_tensor_slices((val_data.values, val_label_neu.values.astype(dtype=np.float32)))
val_dataset_agr = tf.data.Dataset.from_tensor_slices((val_data.values, val_label_agr.values.astype(dtype=np.float32)))
val_dataset_con = tf.data.Dataset.from_tensor_slices((val_data.values, val_label_con.values.astype(dtype=np.float32)))
val_dataset_opn = tf.data.Dataset.from_tensor_slices((val_data.values, val_label_opn.values.astype(dtype=np.float32)))

test_dataset_ext = tf.data.Dataset.from_tensor_slices((test_data.values, test_label_ext.values.astype(dtype=np.float32)))
test_dataset_neu = tf.data.Dataset.from_tensor_slices((test_data.values, test_label_neu.values.astype(dtype=np.float32)))
test_dataset_agr = tf.data.Dataset.from_tensor_slices((test_data.values, test_label_agr.values.astype(dtype=np.float32)))
test_dataset_con = tf.data.Dataset.from_tensor_slices((test_data.values, test_label_con.values.astype(dtype=np.float32)))
test_dataset_opn = tf.data.Dataset.from_tensor_slices((test_data.values, test_label_opn.values.astype(dtype=np.float32)))

In [14]:
embedding = "https://tfhub.dev/google/nnlm-id-dim128/2"

hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)

model_ext = tf.keras.Sequential([
        hub_layer,
        tf.keras.layers.Dense(16, activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1, activation='sigmoid')])

# model_ext = tf.keras.Sequential([
#     tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
#     tf.keras.layers.Conv1D(128, 5, activation='relu'),
#     tf.keras.layers.GlobalMaxPooling1D(),
#     tf.keras.layers.Dense(24, activation='relu'),
#     tf.keras.layers.Dense(1, activation='sigmoid')
# ])

In [15]:
batch_size = 128
num_examples = tf.data.experimental.cardinality(train_dataset_ext).numpy()
ext_batches = train_dataset_ext.shuffle(num_examples // 4).batch(batch_size).prefetch(1)
val_batches = val_dataset_ext.batch(batch_size).prefetch(1)

In [16]:
model_ext.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='binary_crossentropy',
              metrics=['accuracy'])

history = model_ext.fit(ext_batches,
                        validation_data=val_batches,
                        epochs=15)

Epoch 1/15
19/19 [==============================] - 24s 1s/step - loss: 0.7007 - accuracy: 0.4981 - val_loss: 0.6933 - val_accuracy: 0.5000
Epoch 2/15
19/19 [==============================] - 22s 1s/step - loss: 0.6888 - accuracy: 0.5318 - val_loss: 0.6960 - val_accuracy: 0.5161
Epoch 3/15
19/19 [==============================] - 23s 1s/step - loss: 0.6801 - accuracy: 0.5399 - val_loss: 0.6912 - val_accuracy: 0.5484
Epoch 4/15
19/19 [==============================] - 23s 1s/step - loss: 0.6650 - accuracy: 0.5860 - val_loss: 0.6860 - val_accuracy: 0.5968
Epoch 5/15
19/19 [==============================] - 22s 1s/step - loss: 0.6447 - accuracy: 0.6265 - val_loss: 0.6828 - val_accuracy: 0.5968
Epoch 6/15
19/19 [==============================] - 23s 1s/step - loss: 0.6143 - accuracy: 0.6735 - val_loss: 0.6719 - val_accuracy: 0.5645
Epoch 7/15
19/19 [==============================] - 23s 1s/step - loss: 0.5811 - accuracy: 0.7132 - val_loss: 0.6716 - val_accuracy: 0.5806
Epoch 8/15
19/19 [==

In [19]:
import tempfile
import os

MODEL_DIR = tempfile.gettempdir()
version = 1
export_path = os.path.join(MODEL_DIR, str(version))
print('export_path = {}\n'.format(export_path))

tf.keras.models.save_model(
    model_ext,
    export_path,
    overwrite=True,
    include_optimizer=True,
    save_format=None,
    signatures=None,
    options=None
)

print('\nSaved model:')
!ls -l {export_path}

export_path = /tmp/1

INFO:tensorflow:Assets written to: /tmp/1/assets


INFO:tensorflow:Assets written to: /tmp/1/assets



Saved model:
total 140
drwxr-xr-x 2 root root   4096 May 26 14:39 assets
-rw-r--r-- 1 root root   7484 May 26 14:39 keras_metadata.pb
-rw-r--r-- 1 root root 125444 May 26 14:39 saved_model.pb
drwxr-xr-x 2 root root   4096 May 26 14:39 variables


In [20]:
!saved_model_cli show --dir {export_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['__saved_model_init_op']:
  The given SavedModel SignatureDef contains the following input(s):
  The given SavedModel SignatureDef contains the following output(s):
    outputs['__saved_model_init_op'] tensor_info:
        dtype: DT_INVALID
        shape: unknown_rank
        name: NoOp
  Method name is: 

signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['keras_layer_1_input'] tensor_info:
        dtype: DT_STRING
        shape: (-1)
        name: serving_default_keras_layer_1_input:0
  The given SavedModel SignatureDef contains the following output(s):
    outputs['dense_3'] tensor_info:
        dtype: DT_FLOAT
        shape: (-1, 1)
        name: StatefulPartitionedCall_2:0
  Method name is: tensorflow/serving/predict
W0526 14:39:38.152416 140121914361728 deprecation.py:506] From /usr/local/lib/python2.7/dist-packages/tensorflow_cor

In [21]:
MODEL_DIR

'/tmp'